In [ ]:
from Global import *            # Global vairables (symbolic)
from BaseTF import BaseTransferFunction
from Experiment import SymbolixExperimentCG
from Utils import FileSave
import pickle

# Loading test Runs

In [ ]:
file = "../Runs/TIA_simple/simple_Z1_Z2_ZL_object"  # Ensure full file path
try:
    with open(file, 'rb') as f:
        experiment = pickle.load(f)
    print("Object loaded successfully.")
except FileNotFoundError:
    print(f"File not found: {file}")
except pickle.UnpicklingError:
    print(f"Error unpickling file: {file}")


# Automated Test Bench
transmissionMatrix =
    - "simple"          : 
    - "symbolic"        : 
    - "some_parasitic"  : 
    - "full_parasitic"  :

In [29]:
_output = [Vop, Von]
_input  = [Iip, Iin]
T_type  = "simple"
experimentName = "TIA"
experimentName += "_" + T_type

baseTF = BaseTransferFunction(_output, _input,
                              transmissionMatrixType=T_type)
baseTF.solve()

impedanceKeys = [
            "Z1_ZL",
            "Z2_ZL",
            "Z3_ZL",
            "Z4_ZL",
            "Z5_ZL",
            "Z1_Z2_ZL",
            "Z1_Z3_ZL",
            "Z1_Z4_ZL",
            "Z1_Z5_ZL",
            "Z2_Z3_ZL",
            "Z2_Z4_ZL",
            "Z2_Z5_ZL",
            "Z3_Z4_ZL",
            "Z3_Z5_ZL",
            "Z4_Z5_ZL"
            ]

# Z_arr_list = Z_arr_list[0:2]
impedanceKeys = ["Z1_ZL"]

for i, key in enumerate(impedanceKeys, 1):
    print(f"==> Running the {experimentName} Experiment for {key} ({i}/{len(impedanceKeys)})\n")
    experiment = SymbolixExperimentCG(experimentName, baseTF)
    experiment.computeTFs(comboKey=key)
    #
    experiment.classifier.classifyBiQuadFilters()
    experiment.classifier.summarizeFilterType()

    experiment.reportSummary(experimentName, key)
    experiment.compilePDF()

print(impedanceKeys)
experiment.classifier.impedanceList

====CommonGate====
(1) set up the nodal equation
(2) solved the base transfer function (symbolic [T])
FOUND THE BASE TF
*** --- Solving for T type: simple --- ***
==> Running the TIA_simple Experiment for Z1_ZL (1/1)

combo key = Z1_ZL


Getting the TFs (CG): 100%|██████████| 110/110 [00:02<00:00, 41.45combo/s]


Number of transfer functions found: 110


Computing filter parameters: 100%|██████████| 110/110 [00:00<00:00, 279.14filter/s]


HP : 0
BP : 4
LP : 4
BS : 3
GE : 2
AP : 0
INVALID-NUMER : 3
INVALID-WZ : 4
INVALID-ORDER : 90
LaTeX report generated and saved to: Runs/TIA_simple/TIA_simple_Z1_ZL/TIA_simple_Z1_ZL_summary.tex

=== Compiling the reports to PDF ===
----** compiling Runs/TIA_simple/TIA_simple_Z1_ZL/TIA_simple_Z1_ZL_summary.tex **----
This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./TIA_simple_Z1_ZL_summary.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-02-20>
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2024/texmf-dist/tex/latex/amsmath/amsmath.sty
For additional information on amsmath, use the `?' option.
(/usr/local/texlive/2024/texmf-dist/tex/latex/amsmath/amstext.sty
(/usr/local/texlive/2024/texmf-dist/tex/latex/am

⎡                                                                              ↪
⎢                                                                              ↪
⎢                       ⎛                  1  ⎞  ⎛                     R_L     ↪
⎢(R₁, ∞, ∞, ∞, ∞, R_L), ⎜R₁, ∞, ∞, ∞, ∞, ─────⎟, ⎜R₁, ∞, ∞, ∞, ∞, ──────────── ↪
⎢                       ⎝                C_L⋅s⎠  ⎝                C_L⋅R_L⋅s +  ↪
⎢                                                                              ↪
⎣                                                                              ↪

↪                                                                              ↪
↪                                                                              ↪
↪  ⎞  ⎛                        1  ⎞  ⎛                          1  ⎞  ⎛        ↪
↪ ─⎟, ⎜R₁, ∞, ∞, ∞, ∞, R_L + ─────⎟, ⎜R₁, ∞, ∞, ∞, ∞, L_L⋅s + ─────⎟, ⎜R₁, ∞,  ↪
↪ 1⎠  ⎝                      C_L⋅s⎠  ⎝                        C_L⋅s⎠  ⎜        ↪
↪                          

# Manual Experiment

In [ ]:
_output = [Vop, Von]
_input  = [Iip, Iin]
T_type  = "some_parasitic"
experimentName = "TIA"
experimentName += "_" + T_type

baseTF = BaseTransferFunction(_output, _input,
                              transmissionMatrixType=T_type)
baseTF.solve()

tf = baseTF.baseSymbolicHs
tf


In [ ]:
sympy.Poly(tf)

In [ ]:
numerator = numer(tf)
denominator = denom(tf)

numerator /= Z1*Z3*Z4*ZL
denominator /= Z1*Z3*Z4*ZL
denominator = simplify(denominator)

denominator = simplify(Poly(denominator, [a11, a12, a22, a21]))
denominator

In [ ]:
from datetime import datetime

# Assuming `tf` contains the LaTeX representation of the transfer function


tf = latex(baseTF.baseHs)  # Example transfer function in LaTeX

# LaTeX report template
latex_template = r"""
\documentclass[12pt]{article}
\usepackage{amsmath}
\usepackage{amssymb}

\title{Transfer Function Report}
\author{Generated by Python}
\date{""" + datetime.now().strftime("%Y-%m-%d") + r"""}

\begin{document}

\maketitle

\section*{Transfer Function}
The symbolic transfer function is given as:
\[
""" + tf + r"""
\]


\end{document}
"""

# Write LaTeX content to a file
with open("transfer_function_report.tex", "w") as file:
    file.write(latex_template)

print("LaTeX report generated: transfer_function_report.tex")


In [ ]:
latex(baseTF.baseHs)